In [11]:
from sklearn.feature_extraction import DictVectorizer
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [3]:
import pandas as pd

# Specify the path to your Parquet file
file_path = 'yellow_tripdata_2023-01.parquet'

# Read the Parquet file into a DataFrame
df = pd.read_parquet(file_path)

# Display the DataFrame
num_columns = len(df.columns)

print(num_columns)


19


In [5]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [6]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

# Calculating the standard deviation of the trip durations
duration_std = df['duration'].std()
print(f'The standard deviation of the trip durations is: {duration_std}')

The standard deviation of the trip durations is: 42.59435124195458


In [7]:
# Filter the DataFrame to keep only records where the duration is between 1 and 60 minutes (inclusive)
filtered_df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

# Calculate the fraction of records left after dropping outliers
fraction_remaining = len(filtered_df) / len(df)
print(f'The fraction of records left after dropping outliers is: {fraction_remaining}')

The fraction of records left after dropping outliers is: 0.9812202822125979


In [9]:
# Select only the pickup and dropoff location IDs and cast them to strings
filtered_df['PULocationID'] = filtered_df['PULocationID'].astype(str)
filtered_df['DOLocationID'] = filtered_df['DOLocationID'].astype(str)

# Turn the DataFrame into a list of dictionaries
data_dict = filtered_df[['PULocationID', 'DOLocationID']].to_dict(orient='records')

# Fit a dictionary vectorizer
vec = DictVectorizer(sparse=False)
feature_matrix = vec.fit_transform(data_dict)

# Get the dimensionality of the matrix (number of columns)
num_columns = feature_matrix.shape[1]
print(f'The dimensionality of the feature matrix is: {num_columns}')

/var/folders/v3/t8v18d8j7vd2fgp8d41vxr_80000gn/T/ipykernel_92568/3061884161.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['PULocationID'] = filtered_df['PULocationID'].astype(str)
/var/folders/v3/t8v18d8j7vd2fgp8d41vxr_80000gn/T/ipykernel_92568/3061884161.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['DOLocationID'] = filtered_df['DOLocationID'].astype(str)


The dimensionality of the feature matrix is: 515


In [12]:
vec = DictVectorizer(sparse=False)
feature_matrix = vec.fit_transform(data_dict)

# Define the response variable (duration)
y = filtered_df['duration'].values

# Train a plain linear regression model with default parameters
model = LinearRegression()
model.fit(feature_matrix, y)

# Predict the duration using the training data
y_pred = model.predict(feature_matrix)

# Calculate the RMSE on the training data
rmse = np.sqrt(mean_squared_error(y, y_pred))
print(f'The RMSE on the training data is: {rmse}')
